In [3]:
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry
from tkinter.filedialog import askopenfilename
import pandas as pd
import requests
from datetime import datetime
import numpy as np

dicionario_moedas = requests.get('https://economia.awesomeapi.com.br/json/all').json()
lista_moedas = list(dicionario_moedas.keys())

def pegar_cotacao():
  moeda_selecionada = moeda.get()
  data_selecionada = calendario_moeda.get()
  dia = data_selecionada[:2]
  mes = data_selecionada[3:5]
  ano = data_selecionada[-4:]

  cotacao = requests.get(f'https://economia.awesomeapi.com.br/json/daily/{moeda_selecionada}--BRL/?start_date={ano}{mes}{dia}&end_date={ano}{mes}{dia}').json()
  cotacao = float(cotacao[0]['bid'])
  label_textocotacao['text'] = f"A cotação da {moeda_selecionada} no dia {data_selecionada} foi de: R${cotacao:.2f}"


def selecionar_arquivo():
  caminhoarquivo = askopenfilename(title="Selecione um arquivo em Excel para abrir")
  var_caminhoarquivo.set(caminhoarquivo)
  if caminhoarquivo:
    botao_atualizarcotacoes['state'] = 'active'
    label_arquivoselecionado['text'] = f'Caminho: {caminhoarquivo}'

def atualizar_cotacoes():
  try:
    moedas_df = pd.read_excel(var_caminhoarquivo.get())
    moedas = list(moedas_df.iloc[:, 0]) #Todas as linhas da coluna 1

    data_inicial = calendario_datainicial.get()
    data_final = calendario_datafinal.get()

    dia_inicial = data_inicial[:2]
    mes_inicial = data_inicial[3:5]
    ano_inicial = data_inicial[-4:]

    dia_final = data_final[:2]
    mes_final = data_final[3:5]
    ano_final = data_final[-4:]

    data_editada_save = f'{data_inicial}_{data_final}'.replace('/', '_')

    for moeda in moedas:
      link = f'https://economia.awesomeapi.com.br/json/daily/{moeda}--BRL/?start_date={ano_inicial}{mes_inicial}{dia_inicial}&end_date={ano_final}{mes_final}{dia_final}'
      cotacoes_moedas = requests.get(link).json()

      for cotacao in cotacoes_moedas:
        data = datetime.fromtimestamp(int(cotacao['timestamp']))
        data = data.strftime('%d/%m/%Y')
        preco = float(cotacao['bid'])

        if data not in moedas_df:
          moedas_df[data] = np.nan # cria a coluna da data preenchido com nan. Poderia ser "", mas o np.nan é como se fosse um null
        
        moedas_df.loc[moedas_df.iloc[:, 0] == moeda, data] = preco # localiza a linha da moeda atual do loop
      
    moedas_df.to_excel(f'Cotacao_dia_{data_editada_save}.xlsx', index=False)
    
    label_atualizarcotacoes['text'] = "Arquivo Atualizado com Sucesso"
    label_atualizarcotacoes['bg'] = '#1A6125'
    label_atualizarcotacoes['fg'] = 'white'
  except ValueError:
    label_atualizarcotacoes['text'] = "Selecione um arquivo Excel no Formato Correto"
    label_atualizarcotacoes['bg'] = '#611605'
    label_atualizarcotacoes['fg'] = 'white'
  except Exception as e:
    print(f'Erro:{e}')


janela = tk.Tk()
janela.title('Sistema de cotação de moedas')

titulo1 = tk.Label(text = 'Cotação de 1 moeda especifica', borderwidth=2, relief='solid')
titulo1.grid(column=0, row=0, columnspan=3, sticky='nswe', padx=10, pady=10)

label_selecionar_moeda = tk.Label(text = 'Selecione a moeda:')
label_selecionar_moeda.grid(column=0, row=1,  padx=10, pady=10, sticky='nswe', columnspan=2)
moeda = ttk.Combobox(values=lista_moedas)
moeda.grid(column=2, row=1, sticky='nswe', padx=10, pady=10)

label_selecionar_dia = tk.Label(text = 'Selecione o dia que deseja pegar a cotação:', anchor='e')
label_selecionar_dia.grid(column=0, row=2, padx=10, pady=10, sticky='nswe', columnspan=2)
calendario_moeda = DateEntry(year=2025, locale='pt_br')
calendario_moeda.grid(row=2, column=2, padx=10, pady=10, sticky='nsew')

label_textocotacao = tk.Label(text="")
label_textocotacao.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky='nsew')

botao_pegarcotacao = tk.Button(text='Pegar cotação', command=pegar_cotacao)
botao_pegarcotacao.grid(row=3, column=2, sticky='nsew', padx=10, pady=10)

titulo2 = tk.Label(text = 'Cotação de multiplas moedas', borderwidth=2, relief='solid')
titulo2.grid(column=0, row=4, columnspan=3, sticky='nswe', padx=10, pady=10)

label_selecionararquivo = tk.Label(text='Selecione um arquivo em Excel com as Moedas na Coluna A:')
label_selecionararquivo.grid(row=5, column=0, sticky='nswe', columnspan=2, padx=10, pady= 10)

var_caminhoarquivo = tk.StringVar()

botao_selecionararquivo = tk.Button(text="Clique para Selecionar", command = selecionar_arquivo)
botao_selecionararquivo.grid(row=5, column=2, padx=10, pady=10, sticky='nsew')

label_arquivoselecionado = tk.Label(text='Nenhum arquivo selecionado', anchor='e')
label_arquivoselecionado.grid(row=6, column=0, columnspan=3, padx=10, pady=10, sticky='nsew')

label_datainicial = tk.Label(text="Data Inicial", anchor='e')
label_datafinal = tk.Label(text="Data Final", anchor='e')
label_datainicial.grid(row=7, column=0, padx=10, pady=10, sticky='nsew')
label_datafinal.grid(row=8, column=0, padx=10, pady=10, sticky='nsew')

calendario_datainicial = DateEntry(year=2025, locale='pt_br')
calendario_datafinal = DateEntry(year=2025, locale='pt_br')
calendario_datainicial.grid(row=7, column=1, padx=10, pady=10,  sticky='nsew')
calendario_datafinal.grid(row=8, column=1, padx=10, pady=10, sticky='nsew')

botao_atualizarcotacoes = tk.Button(text='Atualizar Cotações', command=atualizar_cotacoes, state='disabled')
botao_atualizarcotacoes.grid(row=9, column=0, padx=10, pady=10, sticky='nsew')

label_atualizarcotacoes = tk.Label(text="")
label_atualizarcotacoes.grid(row=9, column=1, columnspan=2, padx=10, pady=10, sticky='nsew')

janela.mainloop()

In [81]:
import customtkinter as ctk
from tkcalendar import DateEntry
from tkinter.filedialog import askopenfilename
import pandas as pd

lista_moedas = ['euro', 'dólar', 'btc', 'yen']

def selecionar_arquivo():
    caminhoarquivo = askopenfilename(title="Selecione um arquivo em Excel para abrir")
    var_caminhoarquivo.set(caminhoarquivo)
    if caminhoarquivo:
        label_arquivoselecionado.configure(text=f'Caminho: {caminhoarquivo}')
        botao_atualizarcotacoes.configure(state='active')

def atualizar_cotacoes():
    try:
        moedas_df = pd.read_excel(var_caminhoarquivo.get())
        moedas = list(moedas_df.iloc[:, 0])

        data_inicial = calendario_datainicial.get()
        data_final = calendario_datafinal.get()

        print(moedas)
        print(data_inicial)
        print(data_final)
        
        label_atualizarcotacoes.configure(text="Arquivo Atualizado com Sucesso", fg_color='#1A6125', text_color='white')
    except ValueError:
        label_atualizarcotacoes.configure(text="Selecione um arquivo Excel no Formato Correto", fg_color='#611605', text_color='white')
    except Exception as e:
        print(f'Erro: {e}')

# Criando a janela principal
janela = ctk.CTk()
janela.title('Sistema de cotação de moedas')

titulo1 = ctk.CTkLabel(janela, text='Cotação de 1 moeda específica', fg_color='gray', corner_radius=8)
titulo1.grid(column=0, row=0, columnspan=3, sticky='nswe', padx=10, pady=10)

label_selecionar_moeda = ctk.CTkLabel(janela, text='Selecione a moeda:')
label_selecionar_moeda.grid(column=0, row=1, padx=10, pady=10, columnspan=2)

moeda = ctk.CTkComboBox(janela, values=lista_moedas)
moeda.grid(column=2, row=1, padx=10, pady=10)

label_selecionar_dia = ctk.CTkLabel(janela, text='Selecione o dia que deseja pegar a cotação:')
label_selecionar_dia.grid(column=0, row=2, padx=10, pady=10, columnspan=2)

calendario_moeda = DateEntry(janela, year=2025, locale='pt_br')
calendario_moeda.grid(row=2, column=2, padx=10, pady=10)

botao_pegarcotacao = ctk.CTkButton(janela, text='Pegar cotação')
botao_pegarcotacao.grid(row=3, column=2, padx=10, pady=10)

titulo2 = ctk.CTkLabel(janela, text='Cotação de múltiplas moedas', fg_color='gray', corner_radius=8)
titulo2.grid(column=0, row=4, columnspan=3, padx=10, pady=10, sticky= 'nsew')

label_selecionararquivo = ctk.CTkLabel(janela, text='Selecione um arquivo em Excel com as Moedas na Coluna A:')
label_selecionararquivo.grid(row=5, column=0, columnspan=2, padx=10, pady=10)

var_caminhoarquivo = ctk.StringVar()

botao_selecionararquivo = ctk.CTkButton(janela, text="Clique para Selecionar", command=selecionar_arquivo)
botao_selecionararquivo.grid(row=5, column=2, padx=10, pady=10)

label_arquivoselecionado = ctk.CTkLabel(janela, text='Nenhum arquivo selecionado')
label_arquivoselecionado.grid(row=6, column=0, columnspan=3, padx=10, pady=10)

label_datainicial = ctk.CTkLabel(janela, text="Data Inicial")
label_datafinal = ctk.CTkLabel(janela, text="Data Final")
label_datainicial.grid(row=7, column=0, padx=10, pady=10)
label_datafinal.grid(row=8, column=0, padx=10, pady=10)

calendario_datainicial = DateEntry(janela, year=2025, locale='pt_br')
calendario_datafinal = DateEntry(janela, year=2025, locale='pt_br')
calendario_datainicial.grid(row=7, column=1, padx=10, pady=10)
calendario_datafinal.grid(row=8, column=1, padx=10, pady=10)

botao_atualizarcotacoes = ctk.CTkButton(janela, text='Atualizar Cotações', command=atualizar_cotacoes, state='disabled')
botao_atualizarcotacoes.grid(row=9, column=0, padx=10, pady=10)

label_atualizarcotacoes = ctk.CTkLabel(janela, text="")
label_atualizarcotacoes.grid(row=9, column=1, columnspan=2, padx=10, pady=10)

janela.mainloop()

['EUR', 'USD', 'BTC', 'CAD']
09/06/2025
09/06/2025


In [2]:
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry
from tkinter.filedialog import askopenfilename
import pandas as pd
import requests
from datetime import datetime
import numpy as np

class CotacaoMoedasApp:
    def __init__(self, root):
        self.root = root
        self.root.title('Sistema de cotação de moedas')

        self.dicionario_moedas = requests.get('https://economia.awesomeapi.com.br/json/all').json()
        self.lista_moedas = list(self.dicionario_moedas.keys())

        self.var_caminhoarquivo = tk.StringVar()

        self.criar_widgets()

    def criar_widgets(self):
        # Título
        titulo1 = tk.Label(self.root, text='Cotação de 1 moeda específica', borderwidth=2, relief='solid')
        titulo1.grid(column=0, row=0, columnspan=3, sticky='nswe', padx=10, pady=10)

        # Seleção de moeda
        label_selecionar_moeda = tk.Label(self.root, text='Selecione a moeda:')
        label_selecionar_moeda.grid(column=0, row=1, padx=10, pady=10, sticky='nswe', columnspan=2)

        self.moeda = ttk.Combobox(self.root, values=self.lista_moedas)
        self.moeda.grid(column=2, row=1, sticky='nswe', padx=10, pady=10)

        # Seleção de data
        label_selecionar_dia = tk.Label(self.root, text='Selecione o dia que deseja pegar a cotação:')
        label_selecionar_dia.grid(column=0, row=2, padx=10, pady=10, sticky='nswe', columnspan=2)

        self.calendario_moeda = DateEntry(self.root, year=2025, locale='pt_br')
        self.calendario_moeda.grid(row=2, column=2, padx=10, pady=10, sticky='nsew')

        self.label_textocotacao = tk.Label(self.root, text="")
        self.label_textocotacao.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky='nsew')

        botao_pegarcotacao = tk.Button(self.root, text='Pegar cotação', command=self.pegar_cotacao)
        botao_pegarcotacao.grid(row=3, column=2, sticky='nsew', padx=10, pady=10)

        # Cotação de múltiplas moedas
        titulo2 = tk.Label(self.root, text='Cotação de múltiplas moedas', borderwidth=2, relief='solid')
        titulo2.grid(column=0, row=4, columnspan=3, sticky='nswe', padx=10, pady=10)

        label_selecionararquivo = tk.Label(self.root, text='Selecione um arquivo em Excel com as Moedas na Coluna A:')
        label_selecionararquivo.grid(row=5, column=0, sticky='nswe', columnspan=2, padx=10, pady=10)

        botao_selecionararquivo = tk.Button(self.root, text="Clique para Selecionar", command=self.selecionar_arquivo)
        botao_selecionararquivo.grid(row=5, column=2, padx=10, pady=10, sticky='nsew')

        self.label_arquivoselecionado = tk.Label(self.root, text='Nenhum arquivo selecionado', anchor='e')
        self.label_arquivoselecionado.grid(row=6, column=0, columnspan=3, padx=10, pady=10, sticky='nsew')

        label_datainicial = tk.Label(self.root, text="Data Inicial", anchor='e')
        label_datafinal = tk.Label(self.root, text="Data Final", anchor='e')
        label_datainicial.grid(row=7, column=0, padx=10, pady=10, sticky='nsew')
        label_datafinal.grid(row=8, column=0, padx=10, pady=10, sticky='nsew')

        self.calendario_datainicial = DateEntry(self.root, year=2025, locale='pt_br')
        self.calendario_datafinal = DateEntry(self.root, year=2025, locale='pt_br')
        self.calendario_datainicial.grid(row=7, column=1, padx=10, pady=10, sticky='nsew')
        self.calendario_datafinal.grid(row=8, column=1, padx=10, pady=10, sticky='nsew')

        self.botao_atualizarcotacoes = tk.Button(self.root, text='Atualizar Cotações', command=self.atualizar_cotacoes, state='disabled')
        self.botao_atualizarcotacoes.grid(row=9, column=0, padx=10, pady=10, sticky='nsew')

        self.label_atualizarcotacoes = tk.Label(self.root, text="")
        self.label_atualizarcotacoes.grid(row=9, column=1, columnspan=2, padx=10, pady=10, sticky='nsew')

    def pegar_cotacao(self):
        moeda_selecionada = self.moeda.get()
        data_selecionada = self.calendario_moeda.get()
        dia, mes, ano = data_selecionada[:2], data_selecionada[3:5], data_selecionada[-4:]

        cotacao = requests.get(f'https://economia.awesomeapi.com.br/json/daily/{moeda_selecionada}-BRL/?start_date={ano}{mes}{dia}&end_date={ano}{mes}{dia}').json()
        cotacao = float(cotacao[0]['bid'])

        self.label_textocotacao['text'] = f"A cotação da {moeda_selecionada} no dia {data_selecionada} foi de: R${cotacao:.2f}"

    def selecionar_arquivo(self):
        caminhoarquivo = askopenfilename(title="Selecione um arquivo em Excel para abrir")
        self.var_caminhoarquivo.set(caminhoarquivo)
        if caminhoarquivo:
            self.botao_atualizarcotacoes['state'] = 'active'
            self.label_arquivoselecionado['text'] = f'Caminho: {caminhoarquivo}'

    def atualizar_cotacoes(self):
        try:
            moedas_df = pd.read_excel(self.var_caminhoarquivo.get())
            moedas = list(moedas_df.iloc[:, 0])

            data_inicial = self.calendario_datainicial.get()
            data_final = self.calendario_datafinal.get()
            dia_inicial, mes_inicial, ano_inicial = data_inicial[:2], data_inicial[3:5], data_inicial[-4:]
            dia_final, mes_final, ano_final = data_final[:2], data_final[3:5], data_final[-4:]

            for moeda in moedas:
                link = f'https://economia.awesomeapi.com.br/json/daily/{moeda}-BRL/?start_date={ano_inicial}{mes_inicial}{dia_inicial}&end_date={ano_final}{mes_final}{dia_final}'
                cotacoes_moedas = requests.get(link).json()

                for cotacao in cotacoes_moedas:
                    data = datetime.fromtimestamp(int(cotacao['timestamp'])).strftime('%d/%m/%Y')
                    preco = float(cotacao['bid'])

                    if data not in moedas_df:
                        moedas_df[data] = np.nan 

                    moedas_df.loc[moedas_df.iloc[:, 0] == moeda, data] = preco  

            moedas_df.to_excel(f'Cotacao_dia_{data_inicial}_{data_final}.xlsx', index=False)
            self.label_atualizarcotacoes['text'] = "Arquivo Atualizado com Sucesso"

        except Exception as e:
            print(f'Erro: {e}')

# Criar e rodar a aplicação
if __name__ == "__main__":
    root = tk.Tk()
    app = CotacaoMoedasApp(root)
    root.mainloop()